<b>Accessing the BBC dataset on Google Drive</b></br>
Selecting transcript lines in this section will navigate to timestamp in the video
In this demo, we'll continue working with Hugging Face transformer models and we'll try two other models for text summarization. The first model that we'll use is the Google Pegasus model that has been trained on the C4 dataset, as well as a news dataset which includes the CNN Daily Mail articles. We'll have these models generate summaries using data from a different dataset, the BBC News Summary dataset. Let's get started. Now we need to pip install all of the libraries that we need because we're starting with a new Colab notebook. Once again, I'm using the GPU runtime in this notebook. Now we need to pip install a second library here as well, the sentencepiece library. The Pegasus tokenizer needs sentencepiece. Sentencepiece is not actually a tokenizer, it's a method for selecting tokens from a pre-compiled list, optimizing the tokenization process based on a supplied vocabulary corpus. Once you have the required libraries installed, you may need to restart your kernel. That's pretty straightforward. Here in Colab, go to runtime, restart runtime and your kernel restart should ensure that you have all of the libraries that you need and can use them. As before, let's connect to our account on Hugging Face hub by using notebook login. Notebook login needs the token that we had generated earlier. As you remember, these access tokens are present in your profile. We likely need both read as well as write permissions, so I'm going to copy over token summarization write, paste it into this input box and use that to log in and connect to Hugging Face Hub. I've uploaded the data set that I plan to use for summarization to Google Drive, the Google Drive associated with the same account I used to log in to Colab. drive.mount("/content/drive") will mount my Google Drive root folder to Colab. Connecting to Google Drive will allow us to access the dataset that's present in Google Drive. You'll be asked to log in with your account and authenticate yourself. You'll be asked for permissions to allow Colab to access Google Drive. Click on "Allow", and with that, your Drive folder should be mounted. Next, let's confirm that this dataset is indeed present on Google Drive. You can see under My Drive, BBC, I have the BBCNewsSummary.zip file. We'll need to unzip and extract the contents of this file before we can use it. Here is the path to the zip file on my drive. I'm going to import the zip file library and use the zip_ref.extractall method to extract the contents of the zip. The extracted contents will be placed in the My Drive BBC folder. Once the extraction operation is complete, you should find a new subfolder in here called BBC News Summary. And if you click through, you should find subfolders for news articles and the corresponding summaries. Let's take a look at the news articles. You can see that these news articles are split into different topics or categories. Let's look at one of these. Each article is present as a different text file. You can see that the text files are numbered 001, 002, and so on. The corresponding numbered text files in the summaries folder will contain the summaries for these articles. We'll use those summaries as reference summaries to compute ROUGE scores for the summaries that we get from the models that we've chosen.

Installing required huggingface libraries/modules

In [ ]:
pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=77f64f9fdc311b71bce80e2d37956e15d97668d0048a3df7a38b735eab363abc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00


Logging  to Hugging Face account so model can be shared and uploaded with community.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Mounting the Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


## TODO Recording:

- Create the Google Drive folder behind the scenes
- Open up Google Drive for the logged in user cloud.user@
- Show that you have a BBC folder under MyDrive
- Show the zip file already in that folder

In [ ]:
path_to_zip_file = "/content/drive/MyDrive/BBC/BBCNewsSummary.zip"

Extracting zip files into the drive

In [ ]:
import zipfile
with zipfile.ZipFile(path_to_zip_file, "r") as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/BBC")

## TODO Recording:

- Go to the BBC folder
- Show the subfolders
- Click through "News Articles" and show the categories
- Click through to a category and show the files


In [ ]:
import re

def extract(filepath):
    pattern = r"(\w+)/(\d+)\.txt$"

    category, file_id = re.search(pattern, str(filepath)).groups()

    with open(filepath, "r", encoding = "unicode_escape") as f:
        text = f.read()
        return category, file_id, text

Creating the dataframe of Articles and Summaries

In [ ]:
import pandas as pd
from pathlib import Path

dataset_path = Path("/content/drive/MyDrive/BBC/BBC News Summary/")

articles_data = list(map(extract, dataset_path.glob("News Articles/*/*.txt")))
summaries_data = list(map(extract, dataset_path.glob("Summaries/*/*.txt")))

articles_df = pd.DataFrame(articles_data, columns = ("Category", "ID", "Article"))
summaries_df = pd.DataFrame(summaries_data, columns = ("Category", "ID", "Summary"))

news_summary_df = articles_df.merge(summaries_df, how = "inner", on = ("Category", "ID"))

news_summary_df.head()

,Category,ID,Article,Summary
0,business,001,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...
1,business,002,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...
2,business,003,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...
3,business,004,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ..."
4,business,005,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...


Take a look at an example of Article and its summary

In [ ]:
news_summary_df["Article"].loc[10]

"Ask Jeeves tips online ad revival\n\nAsk Jeeves has become the third leading online search firm this week to thank a revival in internet advertising for improving fortunes.\n\nThe firm's revenue nearly tripled in the fourth quarter of 2004, exceeding $86m (Â£46m). Ask Jeeves, once among the best-known names on the web, is now a relatively modest player. Its $17m profit for the quarter was dwarfed by the $204m announced by rival Google earlier in the week. During the same quarter, Yahoo earned $187m, again tipping a resurgence in online advertising.\n\nThe trend has taken hold relatively quickly. Late last year, marketing company Doubleclick, one of the leading providers of online advertising, warned that some or all of its business would have to be put up for sale. But on Thursday, it announced that a sharp turnaround had brought about an unexpected increase in profits. Neither Ask Jeeves nor Doubleclick thrilled investors with their profit news, however. In both cases, their shares f

In [ ]:
news_summary_df["Summary"].iloc[10]

'Ask Jeeves has become the third leading online search firm this week to thank a revival in internet advertising for improving fortunes.Its $17m profit for the quarter was dwarfed by the $204m announced by rival Google earlier in the week.During the same quarter, Yahoo earned $187m, again tipping a resurgence in online advertising.Neither Ask Jeeves nor Doubleclick thrilled investors with their profit news, however.Ask Jeeves, once among the best-known names on the web, is now a relatively modest player.'

https://huggingface.co/docs/datasets/v1.2.0/loading_datasets.html#from-a-pandas-dataframe
Instantiating a Dataset object from pandas Dataframe.
Note that we are disabling default index,otherwise one addl column will be added as index

In [ ]:
from datasets import Dataset, DatasetDict

news_summary_ds = Dataset.from_pandas(news_summary_df, preserve_index = False)

news_summary_ds

Dataset({
    features: ['Category', 'ID', 'Article', 'Summary'],
    num_rows: 2225
})

Text cleaning function is defined

In [ ]:
def clean_txt(example):
    for txt in ["Article", "Summary"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("\\", " ")
       example[txt]  = example[txt].replace("/", " ")
       example[txt]  = example[txt].replace("\n"," ")
       example[txt]  = example[txt].replace("'s"," ")
       example[txt]  = example[txt].replace('"', ' ')
    return example

Then take a look at an example:

In [ ]:
cleaned_news_summary_ds = news_summary_ds.map(clean_txt)

cleaned_news_summary_ds

Map:   0%|          | 0/2225 [00:00<?, ? examples/s]

Dataset({
    features: ['Category', 'ID', 'Article', 'Summary'],
    num_rows: 2225
})

Observing one example of raw data

In [ ]:
news_summary_ds[0]

{'Category': 'business',
 'ID': '001',
 'Article': 'Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.\n\nThe firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.\n\nTime Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free 

Now observing cleaned up version of the same raw data

In [ ]:
cleaned_news_summary_ds[0]

{'Category': 'business',
 'ID': '001',
 'Article': "ad sales boost time warner profit  quarterly profits at us media giant timewarner jumped 76% to $1.13bn (â£600m) for the three months to december, from $639m year-earlier.  the firm, which is now one of the biggest investors in google, benefited from sales of high-speed internet connections and higher advert sales. timewarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. its profits were buoyed by one-off gains which offset a profit dip at warner bros, and less users for aol.  time warner said on friday that it now owns 8% of search-engine google. but its own internet business, aol, had has mixed fortunes. it lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. however, the company said aol  underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. it hopes to increase subscribers by offering the online service free to timew

Video Transcript for the Below Parts <br/>

<b>Generating summaries using Pegasus</b></br>
Selecting transcript lines in this section will navigate to timestamp in the video
We are now ready to access and use the Google Pegasus model for text summarization. And for that, we need two classes from the Hugging Face Transformers library. PegasusForConditionalGeneration, this will allow us to access the pre-trained model and the PegasusTokenizer will allow us to access the tokenizer for the model. Now the path to the Pegasus model is available here on Hugging Face, google/pegasus-cnn_dailymail. Pegasus models are sequence-to-sequence models with an encoder-decoder model architecture. The pre-training task used for Pegasus is similar to summarization. Important sentences are removed or masked from an input document and are generated together as one output sequence from remaining sentences. So this task is similar to performing an extractive summary. On line 3, we access the tokenizer for the Pegasus model by calling PegasusTokenizer.from_pretrained and parsing in the model path. And in a similar way, on line 4, we access the model itself using PegasusFor ConditionalGeneration.from_pretrained. This will download the model and the tokenizer and make it available here in your Colab notebook. Based on the information on the model card for this particular Pegasus model, this was pre-trained on the C4 dataset as well as a dataset called Huge News. And the Huge News dataset includes the CNN Daily Mail dataset, which we used in the previous demo, which is why we are working with a different dataset in this demo, the BBC Summaries dataset. If you just print out the contents of this tokenizer object, you'll get an overview of what the tokenizer looks like, the special tokens that it uses, the maximum length of the model, which is 1024. The vocabulary for this tokenizer is 96,103. Now let's take a look at the model itself. We simply print out the contents of this model. This is a PyTorch model and, again, a heads up that we'll be using PyTorch to get predictions from this model. You get an overview of the layers in the model. You can see there is a shared embedding layer and then there is an encoder. And if you scroll down below, you'll be able to see the Pegasus decoder. We are now ready to put our Pegasus model through its paces. Let's get it to summarize a single article. This is the article at Index 5. I access this article from the cleaned news summary dataset and I store it in the example text variable. This seems to be an article about movies from the year 2004. All right, we have our article. Let's instantiate our Pegasus pipeline. On line 3, we instantiate the pipeline object, specify summarization as the task that we want to perform, the model is the Pegasus model, truncation = True. So if our input sentences have tokens that are longer than the token length supported by the model, those sentences will be truncated. On line 4, I invoke the summarizer on the example text to get the summary. Notice that I do not need to specify a prefix before the example text because this particular model is meant for summarization and does not perform other tasks. Let's take a look at the summary that this produces. You may see a warning here that says some weights were not initialized from the model checkpoint. This means that if you're using this model on a production task, you should probably fine-tune this model on your dataset before you can use it. I tried fine-tuning this model using the resources available on Colab, the GPU that is made available to us, and basically, I found it very hard. Even a single epoch took several hours. Now, you're familiar with how to fine-tune a model from the previous demo. If you have more resources available to you, I suggest you try fine-tuning this model on a dataset of your choice, maybe the CNN Daily Mail. Here is the summary generated by this model. It seems to be a decent summary, but we'll only know once we compute the rouge score. Let's access the reference text, that is, the summary available along with the dataset itself. This is the reference text and we have the candidate text that is the summary generated by the model. Let's use rouge to evaluate the summary. You'll need to load rouge metrics into this Colab notebook. I call rouge.compute as we've done before, parse in the predictions from the model, the reference text. Let's take a look at the summary result, and you can see that the rouge score is pretty decent. It's 0.46 for rouge1, 0.38 for rouge2, and 0.25 for rougeL and rougeLsum. So in spite of the model weights not being perfectly initialized, the summary that this model generated was a fairly decent one.

## Preprocess

In [ ]:
pip install transformers

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

Pegasus model tokenizer and model are instantiated

In [ ]:
PEGASUS_MODEL = "google/pegasus-cnn_dailymail"

tokenizer = PegasusTokenizer.from_pretrained(PEGASUS_MODEL)
model = PegasusForConditionalGeneration.from_pretrained(PEGASUS_MODEL)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Pegasus tokenizer info can be seen

In [ ]:
tokenizer

PegasusTokenizer(name_or_path='google/pegasus-cnn_dailymail', vocab_size=96103, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>'

Pegasus Model can be observed

In [ ]:
model

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

Applying Model for summarization of a text

In [ ]:
EXAMPLE_TEXT_INDEX = 5

example_text = cleaned_news_summary_ds["Article"][EXAMPLE_TEXT_INDEX]

example_text

'japan narrowly escapes recession  japan  economy teetered on the brink of a technical recession in the three months to september, figures show.  revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. on an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. a common technical definition of a recession is two successive quarters of negative growth.  the government was keen to play down the worrying implications of the data.  i maintain the view that japan  economy remains in a minor adjustment phase in an upward climb, and we will monitor developments carefully,  said economy minister heizo takenaka. but in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead, observers were less sanguine.  it  painting a picture of a recovery... much patchier than previously thought,  said paul sheard, e

Summarisation on text is obtained. Setting truncation=True as some sentences are long enough to raise index error as maximum input sequence length of 1024 is getting exceeded

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model = PEGASUS_MODEL, truncation = True)
summary_txt = summarizer(example_text)

summary_txt

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'summary_text': 'revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter .<n>on an annual basis, the data suggests annual growth of just 0.2% .<n>The government was keen to play down the worrying implications of the data .'}]

Reference summary for that text is also obtained

In [ ]:
ref_txt = cleaned_news_summary_ds["Summary"][EXAMPLE_TEXT_INDEX]

ref_txt

'on an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought.a common technical definition of a recession is two successive quarters of negative growth.revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter.japan  economy teetered on the brink of a technical recession in the three months to september, figures show.'

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

Rouge score for that text summary is obtained

In [ ]:
summary_result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                               references = [ref_txt], use_stemmer = True)
summary_result

{'rouge1': 0.5762711864406779,
 'rouge2': 0.4827586206896552,
 'rougeL': 0.3728813559322034,
 'rougeLsum': 0.3728813559322034}

All article texts and summaries are defined in a list

In [ ]:
article_texts = cleaned_news_summary_ds["Article"]

article_summaries = cleaned_news_summary_ds["Summary"]

Generating summaries(zero shot) for first 50 rows only as it will takes a very long time in summarising whole set of examples

In [ ]:
from tqdm import tqdm

candidate_summaries = []

for i, text in enumerate(tqdm(article_texts[:50])):
    candidate = summarizer(text)
    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [36:13<00:00, 43.47s/it]


Aggregated rouge scores are obtained

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.37882844517155145,
 'rouge2': 0.28027316187960294,
 'rougeL': 0.2876354823978982,
 'rougeLsum': 0.2893569669205097}

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                             use_stemmer = True, use_aggregator = False)

Minimum and maximum Rouge score indices are obtained to check best and worst summaries

In [ ]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rougeLsum"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(15, 48)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,Fourth quarter sales rose 2% to $11.1bn from $...,timewarner said fourth quarter sales rose 2% t...
1,The dollar has hit its highest level against t...,the dollar has hit its highest level against t...
2,Russian state-owned rosneft bought the yugansk...,yukos' owner menatep group says it will ask ro...
3,british airways blames high fuel prices for a ...,"rod eddington, ba chief executive, said the r..."
4,allied domecq shares in London rose 4% by 1200...,pernod has reduced the debt it took on to fund...


Taking a look at the highest and lowest rouge score case's predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(35, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(35, "Reference_summaries"))

Actual Summary
Many foreign firms had protested that the sec was imposing an unfair burden .<n>New rules are the result of the sarbanes-oxley act, part of the us clean-up after corporate scandals .<n> section 404 of the sox act, calls for all firms to certify that their financial reporting is in line with us rules .
Reference Summary
many foreign firms had protested that the sec was imposing an unfair burden.the sec said it had taken into consideration the fact that foreign companies were already working to meet more onerous financial reporting rules in their home countries.section 404 of the sox act, as the legislation is nicknamed, calls for all firms to certify that their financial reporting is in line with us rules.foreign firms have been given an extra year to meet tough new corporate governance regulations imposed by the us stock market watchdog.big us firms already have to meet the requirements, but smaller ones and foreign-based firms which list their shares on us stock markets

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(22, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(22, "Reference_summaries"))

Actual Summary
French economy picks up speed at the end of 2004, official figures show .<n>But still looks set to have fallen short of the government hopes .<n>If confirmed, that would be the best quarterly showing since early 2002 .
Reference Summary
despite the apparent shortfall in annual economic growth, the good quarterly figures - a so-called  flash estimate  - mark a continuing trend of improving indicators for the health of the french economy.also on friday, france  industrial output for december was released, showing 0.7% growth.it leaves gdp up 2.3% for the full year, but short of the 2.5% which the french government had predicted.service sector activity improved in january, hitting a seven-month high.but unemployment remains high at about 10%.


Video Transcript for the Below Parts <br/>
<b>Generating summaries using BART</b></br>
Selecting transcript lines in this section will navigate to timestamp in the video
We are now ready to use the last of the transformer models that we'll work with in this course, the Bart model for text summarization. The BartTokenizer will allow us to load the tokenizer used by Bart and BartForConditionalGeneration will allow us to load the pretrained model. Now, the path to the Bart model is here as you see on screen, facebook/bart-large-cnn. The Bart model is a sequence-to-sequence encoder-decoder model with a bidirectional encoder. Bidirectional encoders were a huge improvement over directional encoders. Directional encoders read the input text sequentially from left to right or right to left. Bidirectional encoders, on the other hand, read the entire sequence of words at once. Even though we call it bidirectional, it's more non-directional. Bart also has an autoregressive decoder, which means the decoder generates text by looking at what text was generated in the previous timesteps of the decoder. This particular checkpoint of the Bart model has been fine-tuned on the CNN Daily Mail dataset, the dataset that we worked with in one of our previous demos, which means it should work very well for summarizing news articles. I've loaded in both the tokenizer as well as the model. Let's print out the contents of the tokenizer just to get an overview of what it's all about. You can see that its vocab size is smaller, 50265. The model max length is 1024, that's how many tokens it will generate for the summary. Let's take a look at the model itself. Let's print out the string representation of the model to see what the layers look like. You can see it has an embedding layer that's the shared layer. It has an encoder, that's the Bart encoder. And if you scroll down below, it has a decoder layer as well. Let's first generate a summary of the example text that we had set up earlier. If you remember, this is an article about movies from 2004. At this point, you should be completely comfortable instantiating the pipelines for your transformer models and then using it to get predictions. I have a pipeline for summarization here. The model is the Bart model, truncation = True. I invoked the summarizer on the example text. Note that I do not need to specify a prefix for this model, this model has been explicitly created for the purpose of summarization. Now, when I look at the summary generated by this model, I can tell at a glance that it's much better than the summary we got from the Pegasus model for the same text. Let's see if the summary is better using objective measures. Let's take a look at the reference summary that we get from the dataset and let's compute the rouge score by comparing the candidate summary from the model with the reference summary from the dataset. And here you can see rouge1 is 0.57. With the Pegasus model, our rouge1 score was around 0.3334, whereas here is 0.57. Rouge2 is also quite high, 0.55. All of the rouge metrics seem to indicate that this particular summarizer is far better, at least, for this article that we've chosen than the Pegasus model that we used previously.

#Loading a  Barttokenizer to process `text` and `summary`:

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

Bart tokenizer and Bart large cnn model are instantiated

In [ ]:
BART_MODEL = "facebook/bart-large-cnn"

tokenizer = BartTokenizer.from_pretrained(BART_MODEL)
model = BartForConditionalGeneration.from_pretrained(BART_MODEL)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
tokenizer

BartTokenizer(name_or_path='facebook/bart-large-cnn', vocab_size=50265, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

In [ ]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

Summarisation on text is obtained. Setting truncation=True as some sentences are long enough to raise index error

In [ ]:
example_text

'japan narrowly escapes recession  japan  economy teetered on the brink of a technical recession in the three months to september, figures show.  revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. on an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. a common technical definition of a recession is two successive quarters of negative growth.  the government was keen to play down the worrying implications of the data.  i maintain the view that japan  economy remains in a minor adjustment phase in an upward climb, and we will monitor developments carefully,  said economy minister heizo takenaka. but in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead, observers were less sanguine.  it  painting a picture of a recovery... much patchier than previously thought,  said paul sheard, e

In [ ]:
summarizer = pipeline("summarization", model = BART_MODEL, truncation = True)

summary_txt = summarizer(example_text)

summary_txt

[{'summary_text': ' revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. on an annual basis, the data suggests annual growth of 0.2%, suggesting a much more hesitant recovery than had previously been thought. A common technical definition of a recession is two successive quarters of negative growth.'}]

In [ ]:
ref_txt

'on an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought.a common technical definition of a recession is two successive quarters of negative growth.revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter.japan  economy teetered on the brink of a technical recession in the three months to september, figures show.'

Rouge score for that text summary using Bart model is obtained

In [ ]:
summary_result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                               references = [ref_txt], use_stemmer = True)
summary_result

{'rouge1': 0.8503937007874016,
 'rouge2': 0.816,
 'rougeL': 0.5826771653543307,
 'rougeLsum': 0.5826771653543307}

Generating summaries(zero shot) for first 50 rows only as it will takes a very long time in summarising whole set of examples

In [ ]:
candidate_summaries = []

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [16:18<00:00, 19.57s/it]


Aggregated rouge scores are obtained

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.40529638452921485,
 'rouge2': 0.28939132860736994,
 'rougeL': 0.29614234654828797,
 'rougeLsum': 0.297175684048496}

Here case by case rouge scores are obtained.

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                             use_stemmer = True, use_aggregator = False)

Minimum and maximum Rouge score indices are obtained to check best and worst summaries

In [ ]:
result_unagg_rsum = np.array(result_unagg["rougeLsum"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(5, 3)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,Timewarner profits up 76% to $1.13bn for the t...,timewarner said fourth quarter sales rose 2% t...
1,Dollar hits its highest level against the euro...,the dollar has hit its highest level against t...
2,State-owned rosneft bought the yugansk unit fo...,yukos' owner menatep group says it will ask ro...
3,High fuel prices hit ba profits british airw...,"rod eddington, ba chief executive, said the r..."
4,Pernod ricard said it was seeking acquisitions...,pernod has reduced the debt it took on to fund...


Taking a look at the highest and lowest rouge score case's predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(10, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(10, "Reference_summaries"))

Actual Summary
Ask jeeves revenue nearly tripled in the fourth quarter of 2004, exceeding $86m (â£46m) doubleclick, one of the leading providers of online advertising, warned that some or all of its business would have to be put up for sale late last year. Neither company thrilled investors with their profit news, however.
Reference Summary
ask jeeves has become the third leading online search firm this week to thank a revival in internet advertising for improving fortunes.its $17m profit for the quarter was dwarfed by the $204m announced by rival google earlier in the week.during the same quarter, yahoo earned $187m, again tipping a resurgence in online advertising.neither ask jeeves nor doubleclick thrilled investors with their profit news, however.ask jeeves, once among the best-known names on the web, is now a relatively modest player.


In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(46, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(46, "Reference_summaries"))

Actual Summary
Ryanair has placed an order for 70 boeing 737-800 planes, in a deal valued at $4bn (â£2.1bn) It also has an option for a further 70 aircraft, a move which brings the ryanair order book up to 225 firm orders and options on 193 more.
Reference Summary
 with this new order and new pricing in place, ryanair expects that unit operating costs (excluding fuel) will continue to fall each year for the next five years,  he added.at the end of this year, ryanair will have taken delivery of about 100 new planes, while the 70 new orders are due for delivery between 2008 and 2012.budget airline ryanair has placed an order for 70 boeing 737-800 planes, in a deal valued at $4bn (â£2.1bn) which should lead to 2,500 new ryanair jobs.ryanair said the new planes would help it to cut operating costs further.ryanair chairman david bonderman said that the 737-800 had  significantly reduced our unit operating costs and allowed us to reduce air fares each year for the last five years .


Computing ROUGE metrics for a set of summaries
Selecting transcript lines in this section will navigate to timestamp in the video
Let's do the same thing we did with the previous Pegasus model. I'm going to take the first 50 articles from our BBC Summaries dataset and I'm going to generate summaries for all 50 articles using this Bart model. We'll append each candidate summary generated into this candidate summaries list. Once we have those, we'll compute the aggregate rouge scores for all 50 candidate summaries. This will give us a better picture of how this model performs versus the Pegasus model. Here are the aggregated results. And just for reference, I'm going to show you the results from the Pegasus model. Observe that the rouge1 score has improved to 0.39. It was previously 0.33. Rouge2 is 0.27 for the new model, it was 0.23 for the Pegasus model. The rougeL and rougeLsum scores have also improved. Overall, the Bart model works much better on our dataset compared with the Pegasus model. Let's get the unaggregated rouge scores for the individual text summaries that were generated and then let's get the indices of the summary with the best rougeLsum score and the worst rougeLsum score. The best candidate summary seems to be at Index 8 and the worst at Index 46. Now, in order to view the summaries versus the reference summaries, let's get both of them in a DataFrame, predicted summaries and reference summaries. Let's take a look at the summary with the best rougeLsum score. This was at Index 8. So I'm going to quickly update the indices here so that we look at the actual summary and the reference summary at Index 8, and a quick look tells you that yes, the summary seems very close to the original. You can see that there is a lot of overlap between words in the original and words in the summary. Let's look at the summary with the worst rougeLsum score. This was the one at Index 46. And here, just a glance tells you there is not a lot of overlap between words in the candidate summary and the words in the reference text.